In [1]:
import torch
import os
import sys

from robot_world.utils.video_diffusion import VideoDiffusion
from robot_world.dataloader.dit_dataloader import create_diffusion_dataloader

/home/kangrui/miniconda3/envs/oasis/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
2024-12-31 10:03:04.512790: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-31 10:03:04.521336: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735660984.530129 3027906 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:173566098

In [2]:
vd=VideoDiffusion()

In [3]:
dataloader=create_diffusion_dataloader(batch_size=2, 
                                       num_workers=2, 
                                       shuffle=False,
                                       data_dir="/home/kangrui/projects/world_model/droid-debug",
                                       dataset_name="droid_100",
                                       image_size=(360,640))

2024-12-31 10:03:11.435979: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2024-12-31 10:03:12.223466: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
batch=next(iter(dataloader))

2024-12-31 10:03:13.139183: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-12-31 10:03:13.171098: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [5]:
prev_frames=batch["prev_frames"]
delta_action=batch["delta_action"]
cur_frame=batch["current_frame"]
print(prev_frames.shape, delta_action.shape, cur_frame.shape)

torch.Size([2, 7, 3, 360, 640]) torch.Size([2, 7]) torch.Size([2, 3, 360, 640])


In [6]:
vd.load_action_bins("/home/kangrui/projects/world_model/robot-world/binning_analysis/bins_uniform.json")

In [7]:
pred_images=vd.ddim_sample(prev_frames, delta_action)

/home/kangrui/projects/world_model/robot-world/src/robot_world/utils/action_binner.py:116: UserWarning: torch.searchsorted(): input value tensor is non-contiguous, this will lower the performance due to extra data copy when converting non-contiguous tensor to contiguous, please use contiguous input value tensor if possible. This message will only appear once per program. (Triggered internally at ../aten/src/ATen/native/BucketizationUtils.h:32.)
  indices = torch.bucketize(delta_actions[:, dim], self.bin_edges[dim]) - 1


In [8]:
pred_images.shape

torch.Size([2, 360, 640, 3])

In [10]:
vd.visualize(prev_frames, delta_action, save_path="./test.png")

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0598,  ..., 0.6023, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.8302,  ..., 0.2234, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.4097,  ..., 0.0386, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.1152,  ..., 0.0000, 0.0000, 0.

In [12]:
t = torch.randint(
                    0, 1000, (batch['current_frame'].shape[0],),
                    device=vd.device
                )

In [13]:
vd.device

device(type='cuda')

In [15]:
rst=vd.process_batch(batch,t)
print(rst['eps_theta'].shape)
print(rst['eps'].shape)

torch.Size([2, 1, 16, 18, 32])
torch.Size([2, 1, 16, 18, 32])
